In [53]:
from utils import *

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

In [54]:
# Bunny point cloud
path_to_bunny = "D:\\Skole\\Semester 10\\Prosjektoppgave\\Data\\bunny\\reconstruction\\bun_zipper.ply"
# path_to_bunny = "D:\\Runar\\Documents\\1_Studier\\Semester_10\\Prosjektoppgave\\22v-TPK4560-Prosjektoppgave-Pose_estimation\\Data\\bunny\\data\\bunny000.ply"
bunny_ply = o3d.io.read_point_cloud(path_to_bunny)
bunny = np.asarray(bunny_ply.points)
bunny_true = np.copy(bunny)
n = bunny.shape[0]

In [55]:
# Correct rotation
rot = np.array([np.pi/3,
              np.pi/3,
              np.pi/3
            ])
# Noise and outliers
noise_1 = np.array([np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05)
                    ])
noise_2 = np.array([np.pi/3 + np.random.normal(0,0.5),
                    np.pi/3 + np.random.normal(0,0.5),
                    np.pi/3 + np.random.normal(0,0.5)
                    ])
outlier = np.array([np.pi*np.random.rand(),
                    np.pi*np.random.rand(),
                    np.pi*np.random.rand()
                    ])
      

In [56]:
# Generating translated cloud with inaccuracies
for i, p in enumerate(bunny):
    if i % 4 == 0:
        bunny[i] = expso3(rot) @ p
        continue
    if i % 4 == 1:
        bunny[i] = expso3(noise_1) @ p
        noise_1 = np.array([np.pi/3 + np.random.normal(0,0.005),
                    np.pi/3 + np.random.normal(0,0.005),
                    np.pi/3 + np.random.normal(0,0.005)
                    ])
        continue
    if i % 4 == 2:
        bunny[i] = expso3(noise_2) @ p
        noise_2 = np.array([np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05),
                    np.pi/3 + np.random.normal(0,0.05)
                    ])
        continue
    if i % 4 == 3:
        bunny[i] = expso3(outlier) @ p
        outlier = np.array([np.pi*np.random.rand(),
                    np.pi*np.random.rand(),
                    np.pi*np.random.rand()
                    ])
        continue

In [57]:
# GNC Initialization
# Initial rotation guess
R0 = np.identity(3)

# Initial loss function data
r = np.zeros(n)
for i in range(n):
    r[i] = np.linalg.norm(bunny_true[i] - R0 @ bunny[i])
r0_max = np.max(r)

# gnc
eps = 0.011 
mu_update_factor = 1.4
max_iterations = 1000
w = np.ones(n)
mu = eps**2 / (2*r0_max**2 - eps**2)

In [58]:
last_iter = []
R_iter = [np.sum(r)]
iterations = 0
for i in range(max_iterations):
    iterations += 1
    last_iter.append(np.sum(w))
    # Weighted Procrustes
    H = bunny.T @ np.diag(w) @ bunny_true
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ np.diag([1,1,np.linalg.det( Vt.T @ U.T)]) @ U.T

    # Loss function
    for j in range(n):
        r[j] = np.linalg.norm(bunny_true[j] - R @ bunny[j])
        w[j] = w_from_r(r[j], eps, mu)

    R_iter.append(np.sum(r))

    mu = mu_update_factor * mu

    if i >= 5:
        if np.sum(w) == last_iter[i]:
            break

In [59]:
print(R)
print(expso3(rot))

[[ 0.17  0.97 -0.15]
 [-0.15  0.17  0.97]
 [ 0.97 -0.15  0.17]]
[[ 0.17 -0.15  0.97]
 [ 0.97  0.17 -0.15]
 [-0.15  0.97  0.17]]


In [60]:
inliers = []
outliers = []

for i, n in enumerate(w):
    if w[i] == 1.0:
        inliers.append(n)
    else:
        outliers.append(n)

percentage = (1-len(inliers)/(len(outliers)+len(inliers)))*100

print("Inliers:\t", len(inliers), "\nOutliers:\t", len(outliers),\
    "\nPercentage:\t",percentage, "%")

Inliers:	 25721 
Outliers:	 10226 
Percentage:	 28.44743650374162 %


In [61]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(bunny)
o3d.visualization.draw_geometries([pcd], width=1600, height=900)

In [62]:
print(last_iter)

[35947.0, 569.77523455695, 3246.114358628335, 11229.709697020113, 15829.051074903296, 16600.471406777975, 17309.83604212679, 17993.095329236592, 18637.39479509606, 19235.92696325349, 19788.835396001406, 20308.639484090138, 20799.56428122825, 21274.172277457757, 21745.309551139057, 22220.756385222212, 22702.97192562146, 23180.980081035737, 23640.792773416204, 24068.376611995103, 24458.038388939465, 24792.916379000577, 25064.791448787433, 25278.768691038284, 25433.003936323712, 25543.862878372165, 25617.253653637315, 25662.40165487585, 25688.239633687896, 25704.648920760555, 25713.56354048398, 25717.3303618211, 25719.572936926597, 25720.510053322585, 25720.471951828204, 25719.894669219007, 25720.426723145814, 25719.99298363397, 25719.8423021156, 25720.032854128858, 25720.22123277743, 25720.301877667036, 25720.71548474326, 25721.065209097425, 25721.2240737084, 25721.224594556195, 25721.115506826085, 25721.0]
